In [349]:
# imports necessary packages
import re
import os
import pandas as pd
from datetime import datetime, timedelta
import statistics as st

# set working directory
os.chdir('D:\Faith and Religion Stuff\Come, Follow Me')

### 18 Sep 2024

As I was studying the scriptures this morning, a few ideas came to me that I wanted to record so I could get back to my study ASAP and finish it before getting engaged in all of this:

* I want to implement a user input aspect to get the start and end dates. I can use Regex to determine whether or not the inputted date is satisfactory for my function.
* This calculator will actually be two calculators - the first will gather information about the start and end reading dates, and determine how many days need to be spent on each talk, and assign dates for each readthrough. **NOTE**: Something I'll need to also account for is whether or not I want to read multiple times. 
* The second calculator will break down each talk into sections of paragraphs based on the number of days that will be spent reading it. 

### 27 Sep 2024

Having dealt with Github desktop giving me problems (it decided out of the blue to stop having access to my D:// drive), with working through the "Dealing with Dates" module twice (once before Github desktop started giving me problems, and then once after I fixed those problems after deleting the work I had already done on the module), I'm now ready to start working on my General Conference calculator. I have:
* the talks from the most recent conference
* a list of information about all the talks
* an understanding of working with dates

I'm hoping that there isn't anything else I'll need to get this project really going now. 

In [350]:
apr_2024_info = pd.read_csv('apr_2024_gc_info.csv')
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146


**NOTE:** Looking the table over, I noticed that my function assigning a day weight of +1 to President Russell M. Nelson's talk did not work properly, so that's something that I adjust for here, and that I will have to fix in my conference calculator later. 

In [351]:
apr_2024_info.at[31,'day_weight'] = 1
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146


One thing I know I'll need to do is establish an initial start date and end date. These are the first day I want to read any portion of any talk from this conference and the last day I want to read any portion of any talk from this conference. These two dates will help me calculate how many days should be spent on each talk, subtracting the ini_start_date from the ini_end_date will tell me how many total days I have to work with. 

In [352]:
ini_start = str(input("What day would you like to start reading?"))
ini_end = str(input("What day would you like to stop reading?"))

ini_start_date = pd.to_datetime(ini_start,format='%m/%d/%Y')
ini_end_date = pd.to_datetime(ini_end,format='%m/%d/%Y')
ini_total_days = ini_end_date - (ini_start_date - timedelta(days=1))
ini_total_days

Timedelta('177 days 00:00:00')

I set the cell above up to take a user input for the start and end dates, to save those, and then to convert those strings to datetime objects. Then I created a new object (ini_total_days) which is a timedelta object that shows the number of days between my start date and end date. In order to ensure that my calculator includes the start date, I had to subtract one day from the initial start date (`ini_start_date`). The resulting `Timedelta` object is 177 days. 

Having that 177 calculation, I can now calculate how many days each talk gets. When I initially did this using a Google Sheet, I did it by first establishing how many times I wanted to read each talk. Then, I established, based on the 177 days I had, I determined how many days would be allotted to each readthrough. From there, I allotted each talk a number of days based on the percentage of total speaking time that talk took to deliver, and added days to talks here and there to make sure there was no remainder of days.

For the sake of consistency, for this calculator I will assume, as I did before, that I will read each talk twice except President Nelson's, which I decided to read 3 times. However, I'll also want to get a user input to establish a read weight column for this calculation. 

In order to account for a possible remainder of days I'll need to establish some sort of standard method for my calculation to follow so I don't have to choose which talks get the remainder days. 

In the cell below, I first establish a read weight (ie, the number of times I want to read each talk) based on role code and organization. Basically, I tell python to look at all of the unique role codes (which represent the organization to which the speaker belongs), then ask the user how many times they want to read talks given by speakers from that organization, and saves that information to a dataframe that then gets merged into the original conference info dataframe in a way that ensures that each talk gets a read weight according to the organization the speaker belongs to. 

The reason I want to include user input when determining the read weight is because I may not want to read every talk more than once in the future, or I may want to read the entire First Presidency's talks more frequently than the others, or something like that. I want the flexibility of being able to specify in the future which organizations' talks I want to read more or less frequently, without having to adjust my code. 

In [353]:
# Initialize an empty read_weights list
read_weights = []

# for every unique role code in the role_code column of the apr_2024_info dataframe...
for item in apr_2024_info.role_code.unique():
    # ... ask the user what the read weight should be and...
    read_weight = int(input(f"How many times would you like to read talks given by the {item}?"))
    # ... save both the role code and the read weight to the read_weights list
    read_weights.append({'role_code':item, 'read_weight':read_weight})

# convert the read_weights list to a dataframe, save with the same name to replace the old item
read_weights = pd.DataFrame(read_weights)

# left-merge the read_weights dataframe to the apr_2024_info dataframe useing the role_code columns as a guide for merging
# left-merge keeps everything in the dataframe being merged to, and only merges data from the second dataframe that has a corresponding value in the original dataframe
apr_2024_info = apr_2024_info.merge(read_weights, on='role_code',how='left')

# display the updated dataframe
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2


In the cell below, I just calculate and display how many talks, paragraphs, and lines I'll need to read, given that I'll be reading each talk multiple times. The total_time also just calculates how many seconds of speaking I'll be reading. It doesn't make much sense, but I'll use it tomorrow or Monday. 

In [354]:
total_talks = apr_2024_info.Title.nunique()
total_time = sum(apr_2024_info['time'])
total_paragraphs = sum(apr_2024_info['paragraphs'])
total_lines = sum(apr_2024_info['lines'])

print(f'Total talks to read: {total_talks}\n'
      f'Total time: {round(total_time/60)} minutes\n'
      f'Total paragraphs: {total_paragraphs}\n'
      f'Total lines: {total_lines}')

Total talks to read: 32
Total time: 415 minutes
Total paragraphs: 1006
Total lines: 5372


### 30 Sep 2024

Now that I have established how many times I want to read each talk, I can create a measurement of how many days I need to spend on each talk. 

I started by creating a column that gives each talk a proportional "weight" to each talk. This "weight" consists of the average of the following four things:
* 1/32 - There were 32 talks given, and this gives each talk equal weight. 
* The length (in seconds) of the talk divided by the length (in seconds) of the total conference. This gives longer talks more weight. 
* The length (in lines) of the talk divided by the length (in lines) of the total conference. This gives talks with more lines more weight. 
* My personal preference, which I asked for the input on above. In this case, that gives the prophet's talk a little more weight. 

In [355]:
apr_2024_info['conf_cons'] = (
    ((1/32) +                                                                       # Each talk is 1 of 32 given, this treats each equally
     apr_2024_info['time']/total_time +                                             # time weight - longer "heavier"
     apr_2024_info['lines']/total_lines +                                           # lines weight - longer "heavier"
     ((apr_2024_info['day_weight'] + 1)/(apr_2024_info['day_weight'] + 1).sum()))   # preference weight - Prophet's talk "heavier"
     / 4                                                                            # Adding each of those and then dividing by 4 gets the average
)

# Gets the sum of the the conference consumption proportions
# The output of 1 means that 100% of the talk is accounted for, and that no talk(s) are being over- or under-weighted
apr_2024_info.conf_cons.sum()

1.0

After creating that ratio, I created a new column, "num_days, that takes the initial number of days calculation and multiplies that number by the conference consumption column. What this ensures is that each talk is assigned a number of days out of the 177 initial days based on the factors I explained above. To reiterate my rational for considering and averaging each of these weights:
* That every talk is inspired and should be treated as equally of worth
* That some speakers were just given less time to speak (by the direction of the Prophet), and should be given proportional attention
* That some talks are just textually longer than others, and that taking that into account is necessary to give each talk equitable treatment
* The fact that, personally, I just value the talks of some offices (in this case, the Prophet's, but that may change in future conferences) over those of other offices

All of those are balanced against each other, and the resulting balance is a percentage that is multiplied by 177 days to determine the number of days that should be spent on each talk. 

In [356]:
apr_2024_info['tot_num_days'] = ini_total_days.days * apr_2024_info['conf_cons']
apr_2024_info['tot_num_days'] = apr_2024_info['tot_num_days'].round()
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7.0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4.0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4.0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7.0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4.0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7.0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7.0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4.0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4.0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4.0


Subtracting the initial number of days from the sum of all the total days in my dataframe yields a result of 0, meaning that if I study each talk for the specified number of days, I'll finish within the time that I expected. It also means that there are no problems with the math I've done so far. It also addresses a problem I anticipated earlier, which is that of needing to tell my calculator what to do with remaining days. Since there are no remaining days, I don't have to worry about that! 

In [357]:
apr_2024_info.tot_num_days.sum() - ini_total_days.days

0.0

### 3 Oct 2024

Now that I have calculated the number of days to spend on each talk, I now need to work on dividing up the number of days per reading based on the number of readings I want to do and the total number of days I want to read each talk. I think the best approach for that will be to first create a new column for each of the readings I could do. 

In the cell below, I first identify the maximum number of readthroughs of a talk. Then, I set up a loop that creates a new column named "Readthrough # _" for every new column that needs to be made ranging from "Readthrough # 1" through the highest number of readthroughs, and I populate each of those columns with NA values. Finally, I replace all the NA values with zero (so they can be added to later), and then display the dataframe to show that everything worked as I wanted it to. 

In [358]:
apr_2024_info['tot_num_days'] = apr_2024_info['tot_num_days'].astype(int)

In [359]:
# find and save the highest number in the read_weights column
max_reads = apr_2024_info.read_weight.max()

# start a loop that, for every number between 1 and whatever the max_reads number is, inclusive...
for i in range(1,max_reads+1):
    # create a new column of NA values titled "Readthrough # _(whatever number the loop is on)_"
    apr_2024_info[f"Readthrough #{i}"] = pd.NA
    
# convert all NA values to "0"
apr_2024_info.fillna(0, inplace=True)

# Display dataframe to show that everything worked
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days,Readthrough #1,Readthrough #2,Readthrough #3
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7,0,0,0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4,0,0,0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4,0,0,0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7,0,0,0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4,0,0,0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7,0,0,0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7,0,0,0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4,0,0,0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4,0,0,0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4,0,0,0


With those columns for each readthrough made, I could move on to creating and applying a function that would take the total number of days and distribute them equally into each column, with earlier readthroughs being given more days if the number of days could not be split evenly. 

My `distribute_days()` function takes a dataframe like the one I've crafted. At minimum, however, it would need to have a column labeled 'tot_num_days', a column labeled 'read_weight', and an appropriate number of columns labeled 'Readthrough #x', where x is highest number in the 'read_weight' column. A dataframe with any number of rows and additional columns would work, so long as those minimum requirements are met. 

The function iterates over every row of the data. Every time it starts working with a row it looks at the 'tot_num_days' column and saves that value as 'total_days', and it saves 'dist_days' (short for distributed days) as 0. Then, so long as 'dist_days' is less than 'total_days', it adds 1 to one of the Readthrough columns. It does this until 'dist_days' is equal to 'total_days', and then it proceeds to the next row, where it changes the total_days to the corresponding value in that row, and resets distributed days to 0 to start the process over. 

After defining the function, I run it on my entire dataframe, and am left with a dataframe that now has the number of days that should be spent on each readthrough. 

In [360]:
def distribute_days(conf_df):
    """
    This function takes a dataframe like the one I have crafted above and distributes the total number of days into the "Readthrough #_" columns.
    """
    # establish that the function needs to repeat for every row of the dataframe
    for index, row in conf_df.iterrows():
        # get total number of days for that talk
        total_days = row['tot_num_days']
        # initialize number of distributed days as 0
        dist_days = 0
        # establish that the function needs to proceed with the following operation until dist_days and total_days are equal
        while dist_days < total_days:
            # for every whole number between 1 and whatever the read_weight (or number of readthroughs) is...
            for i in range(1,row['read_weight']+1):
                # if dist_days is still less than total_days...
                if dist_days < total_days:
                    # add 1 to whatever value is in the "Readthrough #(number between 1 and number of readthroughs)" column and...
                    conf_df.at[index, f'Readthrough #{i}'] += 1
                    # add 1 to dist_days
                    dist_days += 1
                    # go back to add 1 to the next column until dist_days is no longer less than total_days
                # if/when dist_days is equal to total_days
                else:
                    # break the process of adding one to each column, and move on to the next row to start the process over
                    break
    # when everything is done, the output of this function is the same dataframe with all of the updated columns
    return conf_df

# run the function on my dataframe
distribute_days(apr_2024_info)

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days,Readthrough #1,Readthrough #2,Readthrough #3
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7,4,3,0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4,2,2,0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4,2,2,0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7,4,3,0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4,2,2,0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7,4,3,0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7,4,3,0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4,2,2,0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4,2,2,0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4,2,2,0


Having established how many days needed to be spent on each readthrough, I could then move on to generating which paragraphs of each talk need to be read on each day. This process will be far more complicated. 

In order to assure that each day of reading for each reading of a particular talk takes roughly the same amount of time, I'll be first dividing up the number of lines that need to be read each day, and then I'll find the number of paragraphs that corresponds most closely to that number of lines. 

I'll start by calculating the number of lines that need to be read for each talk, each day, each readthrough. First I need to create a bunch of new columns, somewhat like I did above. In order to do that, I just needed to apply 2 iterative loops so that the cell below looks at the highest number of days for each readthrough and then creates a number of columns for each readthrough that corresponds to the highest number of days. These columns come out being name something like 'r1d1' for readthrough 1 day 1 and r3d3 for readthrough 3 day 3. 

In [361]:
columns = ['pk', 'Author', 'role', 'role_code', 'day_weight', 'Title',
       'Description', 'Link', 'time', 'paragraphs', 'lines', 'read_weight',
       'conf_cons', 'tot_num_days', 'Readthrough #1', 'Readthrough #2',
       'Readthrough #3']

apr_2024_info = apr_2024_info[columns]

In [362]:
apr_2024_info

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,lines,read_weight,conf_cons,tot_num_days,Readthrough #1,Readthrough #2,Readthrough #3
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,151,2,0.038572,7,4,3,0
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,164,2,0.022629,4,2,2,0
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,164,2,0.022237,4,2,2,0
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,181,2,0.040570,7,4,3,0
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,137,2,0.021051,4,2,2,0
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,169,2,0.041255,7,4,3,0
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,179,2,0.040637,7,4,3,0
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,157,2,0.021952,4,2,2,0
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,176,2,0.023117,4,2,2,0
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,146,2,0.021600,4,2,2,0


I don't know why, but there was no explanation of the contents of the cell below, neither in markdown nor in comments. 

In the cell below, I establish a series of loops to conditionally create a new set of columns in the conference dataframe. 
* the first loop creates a repetition for every number (i) between 1 and whatever the highest number is in the "read_weight" column. The (i) will be used to designate readthrough number. 
* the second loop creates a repetition for every number (x) between 1 and whatever the highest number of days specified in the "Readthrough #i" column. The (x) will be used to designate the day number of any given readthrough.
* the third loop iterates through every row of the apr_2024_info dataframe. 
* the if condition just states defines that if the number of days assigned to any readthrough is zero (because some talks get more readthroughs than others), it should create new columns and fill them with "NA" values. Otherwise, if a readthrough is give a number of days, it should assign the initial number of lines, start point, and end point as 0, to be filled in later. 

In plainer English, the cell below creates columns for storing or defining a start point, a number of lines, and an end point column for every day of every readthrough, regardless of the number of days or readthroughs.

**NOTE:** I had to come back to the cell below two times. When I originally wrote it, I only created and initialized columns for the number of lines for each day of each readthrough. However, on one occasion I came back to add in a start and end line column, and then towards the end of the project I came back again to also make a paragraph start and end column.

In [363]:


for i in range(1, apr_2024_info['read_weight'].max()+1):
    for x in range(1, apr_2024_info[f'Readthrough #{i}'].max()+1):
        for index, row in apr_2024_info.iterrows():
            if row[f'Readthrough #{i}'] != 0:
                apr_2024_info.at[index, f'r{i}d{x}_l_start'] = int(0)
                apr_2024_info.at[index, f'r{i}d{x}_lines'] = int(0)
                apr_2024_info.at[index, f'r{i}d{x}_l_end'] = int(0)
                apr_2024_info.at[index, f'r{i}d{x}_p_start'] = int(0)
                apr_2024_info.at[index, f'r{i}d{x}_p_end'] = int(0)
            else:
                apr_2024_info.at[index, f'r{i}d{x}_l_start'] = pd.NA
                apr_2024_info.at[index, f'r{i}d{x}_lines'] = pd.NA
                apr_2024_info.at[index, f'r{i}d{x}_l_end'] = pd.NA
                apr_2024_info.at[index, f'r{i}d{x}_p_start'] = pd.NA
                apr_2024_info.at[index, f'r{i}d{x}_p_end'] = pd.NA

apr_2024_info.columns

Index(['pk', 'Author', 'role', 'role_code', 'day_weight', 'Title',
       'Description', 'Link', 'time', 'paragraphs', 'lines', 'read_weight',
       'conf_cons', 'tot_num_days', 'Readthrough #1', 'Readthrough #2',
       'Readthrough #3', 'r1d1_l_start', 'r1d1_lines', 'r1d1_l_end',
       'r1d1_p_start', 'r1d1_p_end', 'r1d2_l_start', 'r1d2_lines',
       'r1d2_l_end', 'r1d2_p_start', 'r1d2_p_end', 'r1d3_l_start',
       'r1d3_lines', 'r1d3_l_end', 'r1d3_p_start', 'r1d3_p_end',
       'r1d4_l_start', 'r1d4_lines', 'r1d4_l_end', 'r1d4_p_start',
       'r1d4_p_end', 'r2d1_l_start', 'r2d1_lines', 'r2d1_l_end',
       'r2d1_p_start', 'r2d1_p_end', 'r2d2_l_start', 'r2d2_lines',
       'r2d2_l_end', 'r2d2_p_start', 'r2d2_p_end', 'r2d3_l_start',
       'r2d3_lines', 'r2d3_l_end', 'r2d3_p_start', 'r2d3_p_end',
       'r2d4_l_start', 'r2d4_lines', 'r2d4_l_end', 'r2d4_p_start',
       'r2d4_p_end', 'r3d1_l_start', 'r3d1_lines', 'r3d1_l_end',
       'r3d1_p_start', 'r3d1_p_end', 'r3d2_l_start', '

Because I found below that some of my columns contained float numbers (numbers with a decimal) rather than integers (whole numbers), I wrote the cell below. I defined a pattern using regular expressions, telling my pattern to look for any non-digit follow by a digit followed by a non-digit followed by any number of digits. Then, I saved the columns that contain float numbers as the object 'float_cols'. I then used the regular expression pattern and function to look through those columns for all the columns that match the pattern I defined. Then I converted only those float columns that matched my pattern into integers. Finally, I display the dtypes to show the datatypes in each column of my dataframe. 

In [364]:
pattern = r'\D\d\D\d+'
float_cols = apr_2024_info.select_dtypes(include=['float'])
change_cols = [col for col in float_cols if re.search(pattern, col)]
apr_2024_info[change_cols] = apr_2024_info[change_cols].astype(int)
apr_2024_info.dtypes

pk              object
Author          object
role            object
role_code       object
day_weight       int64
                 ...  
r3d3_l_start    object
r3d3_lines      object
r3d3_l_end      object
r3d3_p_start    object
r3d3_p_end      object
Length: 72, dtype: object

In [365]:
# failed code - commented out so I can just run all as needed

# def distribute_lines(conf_df):
#     """
#     This function takes a dataframe like the one I have crafted above and distributes the total number of days into the "Readthrough #_" columns.
#     """
#     for index, row in conf_df.iterrows():
#         total_lines = row['lines']
#         dist_lines = 0
#         readthrough = 0

        
#         while dist_lines < total_lines and readthrough <= row['read_weight']:
#             readthrough =+ 1
#             if readthrough > row['read_weight']:
#                 break
#             else:
#                 for i in range(1, row[f'Readthrough #{readthrough}']+1):
#                     if dist_lines < total_lines:
#                         conf_df.at[index, f'r{readthrough}d{i}_lines'] += 1
#                         dist_lines += 1
#                     else:
#                         break
                 

#     return conf_df

# distribute_lines(apr_2024_info)


In [366]:
# failed code - commented out so I can just run all as needed

# def distribute_lines(conf_df):
#     """
#     Distribute the total number of lines into the 'Readthrough #_' columns.
#     """
#     for index, row in conf_df.iterrows():
#         total_lines = row['lines']
#         dist_lines = 0
#         readthrough = 1  # Initialize for the row

#         print(f"Processing row {index}: total_lines = {total_lines}, read_weight = {row['read_weight']}")

#         while dist_lines < total_lines and readthrough <= row['read_weight']:

            

#             for day in range(1, row[f'Readthrough #{readthrough}']+1):
#                 print(f"  Readthrough {readthrough}: max_days_in_readthrough = {row[f'Readthrough #{readthrough}']+1}")
#                 if dist_lines < total_lines:
#                     # Add 1 line to the appropriate day column
#                     conf_df.at[index, f'r{readthrough}d{day}_lines'] += 1
#                     dist_lines += 1
#                     print(f"    Distributed line to r{readthrough}d{day}_lines, dist_lines = {dist_lines}")
#                 else:
#                     print(f"    All lines distributed for this readthrough.")
#                     break  # Stop if we've distributed all the lines
            
#             # After finishing the current readthrough
#             print(f"  Moving to readthrough {readthrough + 1}")
#             readthrough += 1

#         print(f"Finished row {index}: total distributed = {dist_lines}")

#     return conf_df

In [367]:
# failed code - commented out so I can just run all as needed
# distribute_lines(apr_2024_info)

In [368]:
# failed code - commented out so I can just run all as needed
# for i in range(1, apr_2024_info['read_weight'].max()+1):
#     for x in range(1, apr_2024_info[f'Readthrough #{i}'].max()+1):
#         for index, row in apr_2024_info.iterrows():
#             if row[f'Readthrough #{i}'] != 0:
#                 apr_2024_info.at[index, f'r{i}_lines_per_day'] = round(row['lines']/row[f'Readthrough #{i}'])
#             else:
#                 apr_2024_info.at[index, f'r{i}_lines_per_day'] = 0
#         for index, row in apr_2024_info.iterrows():
#             if row[f'r{i}_lines_per_day'] != 0:
#                 if i == 1:
#                     apr_2024_info.at[index, f'r{i}d{x}_lines'] = f"1 - {row[f'r{i}_lines_per_day']}"
#                 elif (x-1)*(row[f'r{i}_lines_per_day']) <= row['lines']:
#                     apr_2024_info.at[index, f'r{i}d{x}_lines'] = f"{(x-1)*(row[f'r{i}_lines_per_day'])} - {x*(row[f'r{i}_lines_per_day'])}"
#                 else:
#                     apr_2024_info.at[index, f'r{i}d{x}_lines'] = 0
            
# apr_2024_info

Having those columns made, I can now fill them with the number of lines. 

Okay, I'm out of time for today. This has been a beast. 

### 4 Oct 2024

Picking up where I left off yesterday, I am currently working on trying to distribute the number of lines across the days in each readthrough. Once those lines have been distributed, I can then incorporate the tables I made that have the text, lines, etc, of each talk. 

In [369]:
def distribute_lines(conf_df):
    """
    This function takes a dataframe like the one I have crafted above and distributes the total number of days into the "Readthrough #_" columns.
    """
    # initialize readthrough count as zero
    readthrough = 0
    # establish maximum number of readthroughs so the loop I create below knows when to end or stop
    max_readthroughs = conf_df['read_weight'].max()
    # start a loop of action that will continue until readthroughs is bigger than max_readthroughs, at which point it will stop
    while readthrough <= max_readthroughs:
        # A - first action: add 1 to readthroughs, establishing which readthrough we are working with
        readthrough += 1
        # B - Check if the current readthrough exists in the DataFrame
        if f'Readthrough #{readthrough}' not in conf_df.columns:
            break  # If the column doesn't exist, exit the loop

        # C - second action: for every row in the dataframe do the following:
        for index, row in conf_df.iterrows():
            # D - second action, cont'd: save the info in the row 'lines' as the total number of lines to be distributed
            total_lines = row['lines']
            # E - second action, cont'd: initialize count of distributed lines as zero
            dist_lines = 0
            # debugging print line - remove later
            print(f"Row {index} - Readthrough {readthrough} has {row[f'Readthrough #{readthrough}']} days.")
            # F - check whether the number of days assigned to a talk in a particular readthrough is 0
            if row[f'Readthrough #{readthrough}'] == 0:
                # debugging print line - remove later
                print(f"Skipping row {index} - Readthrough {readthrough} because it has 0 days.")
                # if that talk has zero days alloted for that readthrough, skip to the next talk
                continue
            
            # G - second action, cont'd: open another while loop that will continue until the number of distributed lines is equal to the number of total lines
            while dist_lines < total_lines:
                # H - first action of second loop: open another loop that operates i number of times, where i is the number of days in the readthrough
                for i in range(1, row[f'Readthrough #{readthrough}']+1):
                    # debugging print line - remove later
                    print(f"Processing row {index}, readthrough {readthrough}, day {i}.")
                    # I - first action of second loop, cont'd: add 1 to the corresponding i day of the current readthrough
                    conf_df.at[index, f'r{readthrough}d{i}_lines'] += 1
                    # J - first action of second loop, cont'd: add 1 to the number of distributed lines
                    dist_lines += 1
                    # debugging print line - remove later
                    print(f"Distributed lines: {dist_lines}/{total_lines}")
                    # K - check if dist_lines is less than total_lines: 
                    if dist_lines >= total_lines:
                        break
                        # Unstated action: If it is less, return to point H and repeat this loop.
                        # If not, end *this* loop and return to point D for the next row.
                
        # L - check if readthroughs is less than or equal to max_readthroughs
        if readthrough > max_readthroughs:
            # If it is, repeat this loop, starting with point A
            break
            # Unstated action: If not, end this loop and go to point M.
    
    # M - return the newly modified dataframe
    return conf_df


In [370]:
distribute_lines(apr_2024_info)

Okay, I finally got that all worked out. The function above basically just makes sure that each day of each readthrough of each talk has a (mostly) equal whole number of lines to be read. From here, over the next few days, I'll need to convert those calculations into the lines at which that day is supposed to end (as the previous day's end line will be the current day's start line, unless it's the first day, in which case the start line will be 1). That's work for another day, though, as I've got work I need to get to.

### 5 Oct 2024

Having succeeded yesterday in getting the number of lines to be read each day of each readthrough of each talk, now I can convert those into endpoints. 

In order to do that in a way that will be applicable no matter the number of readthroughs or days that are spent on each readthrough in future conferences, I need to make use of regular expressions. I will convert the operations below into a function later, but first I'll explain what's happening. 

First, I save the column names in my conference dataframe as a list. 
Then, I establish a pattern for identifying columns that containing the number of verses to be read on the first day of any readthrough, and a pattern for identifying columns containing the number of verses to be read on all other days of any readthrough. 
Next, I use Python's built in list comprehension to create a separate list of each of those columns.
Finally, I employ a conditional statement that prints each list so long as the list exists. (The list will only not exist if no matches are found, which would only happen if there was an error at some previous point of my code.)

In [371]:
# save column names as a list
col_names = list(apr_2024_info.columns)

# establish regular expression patterns
d1_pattern = r'\D\d+\D1'
non_d1_pattern = r'\D\d+\D[2-9]+'

# use list comprehension to create lists of column names matching either pattern
d1_list = [col for col in col_names if re.search(d1_pattern, col)]
non_d1_list = [col for col in col_names if re.search(non_d1_pattern, col)]

# If a list of columns matching the day 1 expression exists, display it
if d1_list:
    print(f'D1 Matches found: {d1_list}')
# Otherwise, state that no matches were found. 
else:
    print('No D1 matches found.')

# If a list of columns matching the non-day 1 expression exists, display it
if non_d1_list:
    print(f'Non-d1 Matches found: {non_d1_list}')
# Otherwise, state that no matches were found. 
else:
    print('No non-d1 matches found.')

D1 Matches found: ['r1d1_l_start', 'r1d1_lines', 'r1d1_l_end', 'r1d1_p_start', 'r1d1_p_end', 'r2d1_l_start', 'r2d1_lines', 'r2d1_l_end', 'r2d1_p_start', 'r2d1_p_end', 'r3d1_l_start', 'r3d1_lines', 'r3d1_l_end', 'r3d1_p_start', 'r3d1_p_end']
Non-d1 Matches found: ['r1d2_l_start', 'r1d2_lines', 'r1d2_l_end', 'r1d2_p_start', 'r1d2_p_end', 'r1d3_l_start', 'r1d3_lines', 'r1d3_l_end', 'r1d3_p_start', 'r1d3_p_end', 'r1d4_l_start', 'r1d4_lines', 'r1d4_l_end', 'r1d4_p_start', 'r1d4_p_end', 'r2d2_l_start', 'r2d2_lines', 'r2d2_l_end', 'r2d2_p_start', 'r2d2_p_end', 'r2d3_l_start', 'r2d3_lines', 'r2d3_l_end', 'r2d3_p_start', 'r2d3_p_end', 'r2d4_l_start', 'r2d4_lines', 'r2d4_l_end', 'r2d4_p_start', 'r2d4_p_end', 'r3d2_l_start', 'r3d2_lines', 'r3d2_l_end', 'r3d2_p_start', 'r3d2_p_end', 'r3d3_l_start', 'r3d3_lines', 'r3d3_l_end', 'r3d3_p_start', 'r3d3_p_end']


The output of the cell above shows that d1_list properly contains a list of all of the r_d1 columns, while the non_d1_list properly contains a list of all of the other r_d_ columns, meaning that it worked exactly the way I wanted it to. 

The reason I need these is so that I can perform specific operations on any number specifically targetted columns, where the targetting is done by finding the names using regular expressions. 

This, unfortunately, was all I had time for today, but I'm happy with what I accomplished in a small amount of time. 

### 6 Oct 2024

Having figured out that I *can* identify columns by what day of any given readthrough they contain information for, I need to figure out how to set up my Regex scan for columns so that it will not require input from me in the future. 

My end goal for this stage of the calculation is to have a set of columns that define the starting and ending line of each day of each readthrough. I can use those starting and ending points for ease of calculating the number of paragraphs to read. 

Simply enough, for all the day ones of each readthrough, I know the starting point is line 1, and I know the ending point is the number of lines to be read that day. I also know that the starting point for every day 2 is the same as the ending point for day 1. The end of day 2, though, and everything after that, gets more complicated. So, I need to create a column for the day 2 end point, and for the start and end point of every day thereafter. I also need to automate that so that my calculator works irrespective of the number of days assigned to each readthrough, which has been the most complicated part of this whole process. 

I went back to the cell above where I created a series of new columns for each day of each readthrough, and added some code to also create, at the same time, empty columns for the start and end points of each readthrough. Tomorrow or the next day, I can populate those start and end points, and then I can get to the next step of my calculator. 

### 8 Oct 2024

Having gone back and created the start and end columns for each day of each readthrough by adding it to the code where I create the columns for the number of lines, my goal today is to populate the line start and line end columns with the appropriate data. 

Starting with day 1 is pretty easy. I just start by defining a regular expression pattern that will catch the line start and line end columns of any readthrough. Specifically, the patterns below will look for column names that:
* start with a literal 'r'
* followed by any number of numbers (meaning it will catch r1, r5, r59, r90987903)
* followed by literal 'd1_l_start', 'd1_lines', or 'd1_l_end'

Only column names that follow those specific patterns are stored in my new lists of columns to be targeted and filled first. Then, I fill all of the day 1 start-line columns with 1, because the starting line on the first day of any readthrough will always be line 1. Then I fill all of the day 1 end-line columns with the information in the day 1 number of lines column, and fill the day 2 start column with the information in the day 1 end column. To exemplify why I do it this way, on my first readthrough of Elder Holland's talk, I'll start on line 1, and read 38 lines, so I'll end the first day on line 38. Then, I'll start day 2 of that readthrough on line 38, where I left off the previous day. It only stays simple up to the start of the second day, though.  

In [372]:
# define my regex patterns
d1_start_pattern = r'r\d+d1_l_start'
d1_lines_pattern = r'r\d+d1_lines'
d1_end_pattern = r'r\d+d1_l_end'
d2_start_pattern = r'r\d+d2_l_start'

# create a list of the column names of the dataframe
cols = list(apr_2024_info.columns)

# initialize columns for storing matching column names as empty lists
d1_l_start_cols = []
d1_lines_cols = []
d1_l_end_cols = []
d2_l_start_cols = []

# establish a loop that looks at all of the column names in the list I made above
for col in cols:
    # if a column name matches the start pattern, put it in the start columns list
    if re.search(d1_start_pattern, col):
        d1_l_start_cols.append(col)
    # if a column name matches the number of lines pattern, put it in that list
    elif re.search(d1_lines_pattern, col):
        d1_lines_cols.append(col)
    # if a column name matches the end pattern, put it in that list
    elif re.search(d1_end_pattern, col):
        d1_l_end_cols.append(col)
    # elif a column name matches the day 2 start pattern, put it in that list
    elif re.search(d2_start_pattern, col):
        d2_l_start_cols.append(col)

# establish a loop that takes all of the column names listed in the start columns list...
for col in d1_l_start_cols:
    # and set the value for every row in that column as 1
    apr_2024_info[col] =  1

# establish a loop that takes the values in all the lines columns...
# and saves those values in the corresponding row of the end lines columns. 
for colx, coly in zip(d1_lines_cols, d1_l_end_cols):
    apr_2024_info[coly] = apr_2024_info[colx]

# establish a loop that takes the values in all the day 1 end line columns...
# and saves those values in the corresponding row of the day 2 start line columns.
for colx, coly in zip(d1_l_end_cols,d2_l_start_cols):
    apr_2024_info[coly] = apr_2024_info[colx]

The code above works for day 1, which is the simplest, because no matter the number of readthroughs I do, I will always have a day one for each readthrough. However, the number of days spent on each readthrough is going to vary based on many factors, including the number of times I choose to readthrough a given conference or any talks in it, when I want to start and finish, the number of talks, whether or not it's a leap year, etc. 

Creating a function to iterate through any number of days is going to be more challenging. What I need and want my code to do is look at whatever the current day is, add the number of lines to the start to get the end line, and save the end line as the start line of the next day until there's not a next day. 

The cell below accomplishes what I want it to. I start off by defining a function that requires a dataframe that has the rows and information I've put into my current dataframe throughout this notebook. As with previous functions, it initializes the readthrough as zero and finds and saves the maximum number of readthroughs. Then it saves the column names to a list for accessing in the function. 

Then, so long as the current readthrough number the function is working with is less than or equal to the maximum number of readthroughs it proceeds. 

The function then adds 1 to the current readthrough count, and checks whether or not there is a column for the current readthrough number establishing the number of days to be spent on that readthrough. If there is not, it breaks the loop. (This is just a redundancy added to assure that the function doesn't try to calculate anything for, say, 5 readthroughs if there are only 4.)

After making that check it looks for the maximum number of days found in the first day column of the first readthrough. Because of how I set my function up earlier, the first readthroughs have more days, so the maximum number of days will be found on the first readthrough. The function then establishes a loop that will iterate as many times as there are maximum reading days. 

For every day of every readthrough, the function using regular expressions to get the names of the start line, number of lines assignment, and end lines of the current day, as well as the start line of the next day and saves them for use in calculations. 

Then, for every row or talk of the dataframe, it does the following:
* Checks to see if this readthrough is assigned any days. If it does, it proceeds to do calculations. If not, it skips to the next row or talk. 
* Saves the start point as 1 if the current day of a readthrough is 1.
* Saves the start point plus the number of lines assigned for that day as the end point.
* Saves today's end point as the start point for the next day. 

There are also a ton of debugging print lines that just print every time an important step is successfully achieved. This helped me to more quickly identify when and where my function was experiencing errors as I put it together. 

In [373]:
# def get_end_lines(conf_df):
#     """
#     This function takes a dataframe that has received all of the manipulations above and calculates the line number at which a reader should start and end given the number of lines assigned to be read on any day of any readthrough of any talk. 
#     """
#     # initialize readthrough count as 0
#     rt = 0
#     # establish maximum number of readthroughs so the loop I create below knows when to end or stop
#     max_rts = conf_df['read_weight'].max()
#     # create a list of the column names in the given dataframe
#     cols = list(conf_df.columns)
#     # start a loop of action that will continue until readthroughs is bigger than max_readthroughs, at which point it will stop
#     while rt <= max_rts:
#         # A - first action: add 1 to readthroughs, establishing which readthrough we are working with
#         rt += 1
#         # debugging line - remove later
#         print(f'Starting readthrough {rt}.')
#         # B - Check if the current readthrough exists in the DataFrame
#         if f'Readthrough #{rt}' not in conf_df.columns:
#             # debugging line - remove later
#             print(f'Readthrough {rt} not found. Function complete.')
#             # if a column for the current readthrough number doesn't exist, exit the loop
#             break

#         # C-1 - Second action: start another loop that does the follow for the highest number of days in whatever readthrough number the function is on
#         for day in range(1, conf_df[f'Readthrough #{rt}'].max()+1):
            
#             # debugging line - remove later
#             print(f'Working through readthrough {rt} day {day}.')

#             # C-2 - set patterns for pulling out info for the start, lines, end, and start of next day
#             rt_start_pattern = fr'r{rt}d{day}_l_start'
#             rt_lines_pattern = fr'r{rt}d{day}_lines'
#             rt_end_pattern = fr'r{rt}d{day}_l_end'
#             rt_next_start_pattern = fr'r{rt}d{day+1}_l_start'

#             # debugging line - remove later
#             print(f'Readthrough {rt} day {day} patterns saved.')

#             # C-3 - initialize variables as empty lists
#             start_col = []
#             lines_col = []
#             end_col = []
#             next_start_col = []

#             # debugging line - remove later
#             print(f'Variables initialized.')

#             # C-4 - search for and save columns defined in patterns
#             for col in cols:
#                 if re.search(rt_start_pattern, col):
#                     start_col.append(col)
#                 elif re.search(rt_lines_pattern, col):
#                     lines_col.append(col)
#                 elif re.search(rt_end_pattern, col):
#                     end_col.append(col)
#                 elif re.search(rt_next_start_pattern, col):
#                     next_start_col.append(col)


#             # debugging line - remove later
#             print(f'Columns found and saved.'
#                   f'Starting column name: {start_col}'
#                   f'Number of lines column: {lines_col}'
#                   f'End column name: {end_col}'
#                   f'Next start column name: {next_start_col}')
            
#             # C-5-a -Start another loop
#             for index, row in conf_df.iterrows():
#                 # C-5-b - check if the number of days assigned for the current readthrough of the current talk is 0
#                 if row[f'Readthrough #{rt}'] == 0:
#                     # debugging print line - remove later
#                     print(f"Skipping row {index} - Readthrough {rt} because it has 0 days.")
#                     # if that talk has zero days alloted for that readthrough, skip to the next talk
#                     continue
                
#                 # unstated action - if the number of days assigned for the current readthrough is greater than zero, proceed to C-5-c

#                 # C-5-c - check if the day number is 1
#                 if day == 1:
#                     # C-5-c-1 - if so, initialize the start column as 1 for every row
#                     conf_df[start_col] = 1
#                     # debugging line - remove later
#                     print(f'Readthrough {rt} day {day} set at 1.')
#                     # C-5-c-1 - then save the end point as the start point (1) plus the number of lines to be read 
#                     # the minus 1 at the end ensures that we end at the assigned reading line, not the line after
#                     for start, read, end in zip(start_col, lines_col, end_col):
#                         conf_df.loc[index, end] = conf_df.loc[index, start] + conf_df.loc[index, read] - 1
#                     # C-5-c-2 - then save today's ending point as the starting point for the next day, to be accessed later
#                     for end, next_start in zip(end_col, next_start_col):
#                         conf_df.loc[index, next_start] = conf_df.loc[index, end]
                
#                 # C-5-d - if the day number IS NOT 1...
#                 else:
#                     # C-5-d-1 - access whatever the start column has already been saved as, add the number of lines, and save that as the end point
#                     for start, read, end in zip(start_col, lines_col, end_col):
#                         conf_df.loc[index, end] = conf_df.loc[index, start] + conf_df.loc[index, read]
#                     # C-5-d-2 - then save today's ending point as the starting point for the next day, to be accessed later
#                     for end, next_start in zip(end_col, next_start_col):
#                         conf_df.loc[index, next_start] = conf_df.loc[index, end]

#             # debugging print line - remove later
#             print(f'Readthrough {rt} columns day {day} saved. Looping back.')
        
#         # debugging print line - remove later
#         print(f'Readthrough {rt} loop finished.')
    
#     return conf_df


# get_end_lines(apr_2024_info)


The cells below are just in place for me to check and demonstrate that my `get_end_lines()` function worked the way I wanted it to. Thankfully it did.

In [374]:
# apr_2024_info.columns

In [375]:
# check_cols = ['r3d1_l_start',
#        'r3d1_lines', 'r3d1_l_end', 'r3d2_l_start', 'r3d2_lines', 'r3d2_l_end',
#        'r3d3_l_start', 'r3d3_lines', 'r3d3_l_end','lines']
# checkset = apr_2024_info[check_cols]
# checkset

Tomorrow I will use the established start and end points to calculate which paragraphs need to be read on which days. Because the paragraphs are **highly** unlikely to line up perfectly with the paragraph ends, I will have to set up a function that finds the paragraph an endpoint lands in and establishes whether it would be closer to the number of lines to be read to include or not include that paragraph, and include or exclude it accordingly. But once I've done that, my final steps will be to get the daily assignments printed out, and then to functionalize every relevant bit of code in this notebook and the initial webscraping notebook, so that when all is said and done all I need to do is plug in the link to the conference landing page, and then everything else gets calculated out for me. 

### 9 Oct 2024

After some contemplation last night, I realized getting date assignments for the conference breakdown would be unnecessary and complicated. It would be unnecessary because as long I follow the pace set out in the breakdown (i.e., reading each talk for the set number of days for each readthrough), then I can read them in whatever order I want, whereas if I focus on needlessly printing out dates, if I ever want to change the order in which I read the talks, my Conference breakdown becomes night unto useless. 

So, I'm going to focus today on just getting the paragraph reading assignments for each day of each readthrough, and then focus on functionalizing all my work into one function through which I can pass the link to any conference. 

Something I need to keep in mind is that, though I'll be working with csv files that I downloaded right now, I probably will not be doing that in the main function. Rather, I'll just be storing each talk as it's own object. Taking that into consideration, my first step today is to import all of the csvs into this notebooks current working memory. I do that in the cell below by:
* defining a file path
* initializing an empty list into which to store the names of the csv files
* looping over each csv file in the folder targeted in the file path and storing its name into the list of csv filenames
* initializing an empty dictionary in which all of the csv files will be stored as separate dataframes
* looping over each csv file in the list of csv filenames, reading it into a pandas dataframe, and storing it in the dictionary of dataframes

In [376]:
# define file path
path_start = 'D:\\Faith and Religion Stuff\\Come, Follow Me\\April 2024 GC Talks\\'

# initialize filenames list
csv_files = []

# get the names of all the csv files in the directory
for file in os.listdir(path_start):
    if file.endswith(".csv"):
        csv_files.append(file)

# initialize a dictionary to store the dataframes
all_talks_df = {}

# import the csv files into pandas dataframes, store each dataframe in the dictionary
for file in csv_files:
    talk = file[:-4]
    all_talks_df[talk] = pd.read_csv(os.path.join(path_start, file))
    print(f'file string: {file}\n'
          f'talk string: {talk}')

# display the dictionary to ensure that everything worked. It did.
all_talks_df   

file string: ad_apr24_lines.csv
talk string: ad_apr24_lines
file string: ams_apr24_lines.csv
talk string: ams_apr24_lines
file string: bhn_apr24_lines.csv
talk string: bhn_apr24_lines
file string: bkt_apr24_lines.csv
talk string: bkt_apr24_lines
file string: dab_apr24_lines.csv
talk string: dab_apr24_lines
file string: dfu_apr24_lines.csv
talk string: dfu_apr24_lines
file string: dgr_apr24_lines.csv
talk string: dgr_apr24_lines
file string: dho_apr24_lines.csv
talk string: dho_apr24_lines
file string: dtc_apr24_lines.csv
talk string: dtc_apr24_lines
file string: ges_apr24_lines.csv
talk string: ges_apr24_lines
file string: gwg_apr24_lines.csv
talk string: gwg_apr24_lines
file string: hbe_apr24_lines.csv
talk string: hbe_apr24_lines
file string: jad_apr24_lines.csv
talk string: jad_apr24_lines
file string: jla_apr24_lines.csv
talk string: jla_apr24_lines
file string: jng_apr24_lines.csv
talk string: jng_apr24_lines
file string: jrh_apr24_lines.csv
talk string: jrh_apr24_lines
file strin

{'ad_apr24_lines':    foreign_key           pk  paragraph_number  \
 0     ad_apr24   ad_apr24_0                 0   
 1     ad_apr24   ad_apr24_1                 1   
 2     ad_apr24   ad_apr24_2                 2   
 3     ad_apr24   ad_apr24_3                 3   
 4     ad_apr24   ad_apr24_4                 4   
 5     ad_apr24   ad_apr24_5                 5   
 6     ad_apr24   ad_apr24_6                 6   
 7     ad_apr24   ad_apr24_7                 7   
 8     ad_apr24   ad_apr24_8                 8   
 9     ad_apr24   ad_apr24_9                 9   
 10    ad_apr24  ad_apr24_10                10   
 11    ad_apr24  ad_apr24_11                11   
 12    ad_apr24  ad_apr24_12                12   
 13    ad_apr24  ad_apr24_13                13   
 14    ad_apr24  ad_apr24_14                14   
 15    ad_apr24  ad_apr24_15                15   
 16    ad_apr24  ad_apr24_16                16   
 17    ad_apr24  ad_apr24_17                17   
 18    ad_apr24  ad_apr24_18    

After importing those dataframes, I realized my life will probably be made much, much easier if each dataframe has a column that lists the running total of lines for each paragraph. In the cell below, I use a for loop to create a new column in each dataframe that gets that running total. 

In [377]:
# iterate over every dataframe stored in the all_talks_df dictionary
for talk, df in all_talks_df.items():
    # create a new column in each dataframe that is the cumulative sum of the number of lines
    df['running_lines'] = df['num_lines'].cumsum()

# display the dataframe to show that everything worked. It did. 
all_talks_df

{'ad_apr24_lines':    foreign_key           pk  paragraph_number  \
 0     ad_apr24   ad_apr24_0                 0   
 1     ad_apr24   ad_apr24_1                 1   
 2     ad_apr24   ad_apr24_2                 2   
 3     ad_apr24   ad_apr24_3                 3   
 4     ad_apr24   ad_apr24_4                 4   
 5     ad_apr24   ad_apr24_5                 5   
 6     ad_apr24   ad_apr24_6                 6   
 7     ad_apr24   ad_apr24_7                 7   
 8     ad_apr24   ad_apr24_8                 8   
 9     ad_apr24   ad_apr24_9                 9   
 10    ad_apr24  ad_apr24_10                10   
 11    ad_apr24  ad_apr24_11                11   
 12    ad_apr24  ad_apr24_12                12   
 13    ad_apr24  ad_apr24_13                13   
 14    ad_apr24  ad_apr24_14                14   
 15    ad_apr24  ad_apr24_15                15   
 16    ad_apr24  ad_apr24_16                16   
 17    ad_apr24  ad_apr24_17                17   
 18    ad_apr24  ad_apr24_18    

Having that squared away, I can now use the primary keys I have in my `apr_2024_info` dataframe to access the information in the dataframes dictionary, including those running totals. What I want to do now is set up a loop or function that will take the endpoint for any given day of a readthrough and find the paragraph closest to that, and set that as the paragraph endpoint for that day of reading. If I can get the end points, I can get the start points no problem, and once I have the start and end points for each day of each readthrough of each talk, I'm done. 

As I started working through this, I realized I had to go back to the point where I initially created a column for the number of lines and add in the creating of a paragraph start and paragraph end column. 

In [378]:
def get_paragraphs(conf_df,talks_dictionary):
    """
    This function assigns starting and ending lines and paragraphs for each day of each readthrough of each talk contained in the conference dataframe.
    """    
    # initialize readthrough count as 0
    rt = 0
    # establish maximum number of readthroughs so the loop I create below knows when to end or stop
    max_rts = conf_df['read_weight'].max()
    # create a list of the column names in the given dataframe
    cols = list(conf_df.columns)
    # start a loop of action that will continue until readthroughs is bigger than max_readthroughs, at which point it will stop
    while rt <= max_rts:
        # A - first action: add 1 to readthroughs, establishing which readthrough we are working with
        rt += 1
        # debugging line - remove later
        print(f'Starting readthrough {rt}.')
        # B - Check if the current readthrough exists in the DataFrame
        if f'Readthrough #{rt}' not in conf_df.columns:
            # debugging line - remove later
            print(f'Readthrough {rt} not found. Function complete.')
            # if a column for the current readthrough number doesn't exist, exit the loop
            break

        # C-1 - Second action: start another loop that does the follow for the highest number of days in whatever readthrough number the function is on
        for day in range(1, conf_df[f'Readthrough #{rt}'].max()+1):
            
            # debugging line - remove later
            print(f'Working through readthrough {rt} day {day}.')

            # C-2 - set patterns for pulling out info for the start, lines, end, and start of next day
            rt_start_l_pattern = fr'r{rt}d{day}_l_start'
            rt_lines_pattern = fr'r{rt}d{day}_lines'
            rt_end_l_pattern = fr'r{rt}d{day}_l_end'
            rt_next_start_l_pattern = fr'r{rt}d{day+1}_l_start'
            rt_start_p_pattern = fr'r{rt}d{day}_p_start'
            rt_end_p_pattern = fr'r{rt}d{day}_p_end'
            rt_next_start_p_pattern = fr'r{rt}d{day+1}_p_start'

            # debugging line - remove later
            print(f'Readthrough {rt} day {day} patterns saved.')

            # C-3 - initialize variables as empty lists
            start_l_col = []
            lines_col = []
            end_l_col = []
            next_start_l_col = []
            start_p_col = []
            end_p_col = []
            next_start_p_col = []

            # debugging line - remove later
            print(f'Variables initialized.')

            # C-4 - search for and save columns defined in patterns
            for col in cols:
                if re.search(rt_start_l_pattern, col):
                    start_l_col.append(col)
                elif re.search(rt_lines_pattern, col):
                    lines_col.append(col)
                elif re.search(rt_end_l_pattern, col):
                    end_l_col.append(col)
                elif re.search(rt_next_start_l_pattern, col):
                    next_start_l_col.append(col)
                elif re.search(rt_start_p_pattern, col):
                    start_p_col.append(col)
                elif re.search(rt_end_p_pattern, col):
                    end_p_col.append(col)
                elif re.search(rt_next_start_p_pattern, col):
                    next_start_p_col.append(col)

            # debugging line - remove later
            print(f'Columns found and saved.'
                  f'Starting column name: {start_l_col}'
                  f'Number of lines column: {lines_col}'
                  f'End column name: {end_l_col}'
                  f'Next start column name: {next_start_l_col}'
                  f'Starting column name: {start_p_col}'
                  f'Number of lines column: {lines_col}'
                  f'End column name: {end_p_col}'
                  f'Next start column name: {next_start_p_col}')
            
            # C-5-a -Start another loop
            for index, row in conf_df.iterrows():
                # C-5-b - check if the number of days assigned for the current readthrough of the current talk is 0
                if row[f'Readthrough #{rt}'] == 0:
                    # debugging print line - remove later
                    print(f"Skipping row {index} - Readthrough {rt} because it has 0 days.")
                    # if that talk has zero days alloted for that readthrough, skip to the next talk
                    continue
                
                # unstated action - if the number of days assigned for the current readthrough is greater than zero, proceed to C-5-c

                # C-5-c - establish the connection between conf_df and talks_dict using the primary key column of the conf_df
                talk = talks_dictionary[f"{conf_df.loc[index,'pk']}_lines"]

                # debugging line - remove later
                print(f'Connecting conf_df to talks_dict using primary key {conf_df.loc[index,"pk"]}.')

                # C-5-d - check if the day number is 1
                if day == 1:
                    # C-5-d-1 - if so, initialize the start column as 1 for every row
                    conf_df[start_l_col] = 1
                    conf_df[start_p_col] = 1
                    # debugging line - remove later
                    print(f'Readthrough {rt} day {day} set at 1.')
                    # C-5-d-1 - then save the end point as the start point (1) plus the number of lines to be read 
                    # the minus 1 at the end ensures that we end at the assigned reading line, not the line after
                    for start, read, end in zip(start_l_col, lines_col, end_l_col):
                        conf_df.loc[index, end] = conf_df.loc[index, start] + conf_df.loc[index, read] - 1
                    # C-5-d-2 - then save today's ending point as the starting point for the next day, to be accessed later
                    for end, next_start in zip(end_l_col, next_start_l_col):
                        conf_df.loc[index, next_start] = conf_df.loc[index, end]
                    # C-5-d-3 - save the end line just calculated as object 'end_line'
                    end_line = conf_df.loc[index, f'r{rt}d{day}_l_end']

                    # debugging line - remove later
                    print(f'Readthrough {rt} day {day} end line set. End line: {end_line}')

                    # C-5-d-4 - find in the talk the paragraph with a running total that is greater than or equal to the end line 
                    end_paragraph_1 = talk.loc[talk['running_lines'] >= end_line].index[0]
                    # C-5-d-5 - find in the talk the paragraph before the one above
                    end_paragraph_2 = talk.loc[talk['running_lines'] >= end_line].index[-1]
                    # C-5-d-6 - determine which paragraph would yield a number of lines being read closer to the target end line and save as end_paragraph
                    if abs(talk.loc[end_paragraph_1,'running_lines'] - end_line) < abs(talk.loc[end_paragraph_2,'running_lines']- end_line):
                        end_paragraph = end_paragraph_1
                    else:
                        end_paragraph = end_paragraph_2
                    # C-5-d-7 - save the selected end_paragraph as the paragraph ending point of the current day of the current readthrough
                    for end_p in end_p_col:
                        conf_df.loc[index, end_p] = end_paragraph
                    
                    for end_p, next_start_p in zip(end_p_col, next_start_p_col):
                        conf_df.loc[index, next_start_p] = conf_df.loc[index, end_p] + 1

                    # debugging line - remove later
                    print(f'Readthrough {rt} day {day} end paragraph set: End paragraph: {end_paragraph}')
                    

                    
                
                # C-5-e - if the day number IS NOT 1...
                else:
                    # C-5-e-1 - access whatever the start column has already been saved as, add the number of lines, and save that as the end point
                    for start, read, end in zip(start_l_col, lines_col, end_l_col):
                        conf_df.loc[index, end] = conf_df.loc[index, start] + conf_df.loc[index, read]
                    # C-5-e-2 - then save today's ending point as the starting point for the next day, to be accessed later
                    for end, next_start in zip(end_l_col, next_start_l_col):
                        conf_df.loc[index, next_start] = conf_df.loc[index, end]
                    # C-5-e-3 - save the end line just calculated as object 'end_line'
                    end_line = conf_df.loc[index, f'r{rt}d{day}_l_end']

                    # debugging line - remove later
                    print(f'Readthrough {rt} day {day} end line set. End line: {end_line}')

                    # C-5-e-4 - find in the talk the paragraph with a running total that is greater than or equal to the end line 
                    end_paragraph_1 = talk.loc[talk['running_lines'] >= end_line].index[0]
                    # C-5-e-5 - find in the talk the paragraph before the one above
                    end_paragraph_2 = talk.loc[talk['running_lines'] >= end_line].index[-1]
                    # C-5-e-6 - determine which paragraph would yield a number of lines being read closer to the target end line and save as end_paragraph
                    if abs(talk.loc[end_paragraph_1,'running_lines'] - end_line) < abs(talk.loc[end_paragraph_2,'running_lines']- end_line):
                        end_paragraph = end_paragraph_1
                    else:
                        end_paragraph = end_paragraph_2
                    # C-5-e-7 - save the selected end_paragraph as the paragraph ending point of the current day of the current readthrough
                    for end_p in end_p_col:
                        conf_df.loc[index, end_p] = end_paragraph

                    for end_p, next_start_p in zip(end_p_col, next_start_p_col):
                        conf_df.loc[index, next_start_p] = conf_df.loc[index, end_p] + 1

                    # debugging line - remove later
                    print(f'Readthrough {rt} day {day} end paragraph set: End paragraph: {end_paragraph}')

            # debugging print line - remove later
            print(f'Readthrough {rt} columns day {day} saved. Looping back.')
        
        # debugging print line - remove later
        print(f'Readthrough {rt} loop finished.')
    
    return conf_df

In [379]:
get_paragraphs(apr_2024_info,all_talks_df)

Starting readthrough 1.
Working through readthrough 1 day 1.
Readthrough 1 day 1 patterns saved.
Variables initialized.
Columns found and saved.Starting column name: ['r1d1_l_start']Number of lines column: ['r1d1_lines']End column name: ['r1d1_l_end']Next start column name: ['r1d2_l_start']Starting column name: ['r1d1_p_start']Number of lines column: ['r1d1_lines']End column name: ['r1d1_p_end']Next start column name: ['r1d2_p_start']
Connecting conf_df to talks_dict using primary key jrh_apr24.
Readthrough 1 day 1 set at 1.
Readthrough 1 day 1 end line set. End line: 0
Readthrough 1 day 1 end paragraph set: End paragraph: 0
Connecting conf_df to talks_dict using primary key jad_apr24.
Readthrough 1 day 1 set at 1.
Readthrough 1 day 1 end line set. End line: 0
Readthrough 1 day 1 end paragraph set: End paragraph: 0
Connecting conf_df to talks_dict using primary key ad_apr24.
Readthrough 1 day 1 set at 1.
Readthrough 1 day 1 end line set. End line: 0
Readthrough 1 day 1 end paragraph se

,pk,Author,role,role_code,day_weight,Title,Description,Link,time,paragraphs,...,r3d2_l_start,r3d2_lines,r3d2_l_end,r3d2_p_start,r3d2_p_end,r3d3_l_start,r3d3_lines,r3d3_l_end,r3d3_p_start,r3d3_p_end
0,jrh_apr24,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,q12,0,Motions of a Hidden Fire,President Holland teaches about the power of p...,https://www.churchofjesuschrist.org/study/gene...,808,21,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,jad_apr24,J. Anette Dennis,First Counselor in the Relief Society General ...,rs,-1,Put Ye On the Lord Jesus Christ,"Sister Dennis teaches about the importance, po...",https://www.churchofjesuschrist.org/study/gene...,716,23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,ad_apr24,Alexander Dushku,Of the Seventy,q70,-1,Pillars and Rays,Elder Dushku teaches us that spectacular spiri...,https://www.churchofjesuschrist.org/study/gene...,677,37,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,us_apr24,Ulisses Soares,Of the Quorum of the Twelve Apostles,q12,0,Covenant Confidence through Jesus Christ,Elder Soares teaches about the importance of l...,https://www.churchofjesuschrist.org/study/gene...,868,19,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,jng_apr24,Jack N. Gerard,Of the Seventy,q70,-1,Integrity: A Christlike Attribute,Elder Gerard teaches that living a life of int...,https://www.churchofjesuschrist.org/study/gene...,684,22,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,hbe_apr24,Henry B. Eyring,Second Counselor in the First Presidency,1p,0,All Will Be Well Because of Temple Covenants,President Eyring teaches that as we make and k...,https://www.churchofjesuschrist.org/study/gene...,992,30,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,dab_apr24,David A. Bednar,Of the Quorum of the Twelve Apostles,q12,0,"“Be Still, and Know That I Am God”","Elder Bednar teaches that when we are “still,”...",https://www.churchofjesuschrist.org/study/gene...,884,29,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,mdf_apr24,Massimo De Feo,Of the Seventy,q70,-1,Rise! He Calleth Thee,Elder De Feo teaches that blessings come when ...,https://www.churchofjesuschrist.org/study/gene...,681,51,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,bhn_apr24,Brent H. Nielson,Of the Presidency of the Seventy,q70,-1,A Record of What I Have Both Seen and Heard,Elder Nielson describes the growth of the Chur...,https://www.churchofjesuschrist.org/study/gene...,709,25,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,jla_apr24,Jose L. Alonso,Of the Seventy,q70,-1,Jesus Christ at the Center of Our Lives,Elder Alonso testifies that as we place Jesus ...,https://www.churchofjesuschrist.org/study/gene...,697,21,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [380]:
apr_2024_info.columns

Index(['pk', 'Author', 'role', 'role_code', 'day_weight', 'Title',
       'Description', 'Link', 'time', 'paragraphs', 'lines', 'read_weight',
       'conf_cons', 'tot_num_days', 'Readthrough #1', 'Readthrough #2',
       'Readthrough #3', 'r1d1_l_start', 'r1d1_lines', 'r1d1_l_end',
       'r1d1_p_start', 'r1d1_p_end', 'r1d2_l_start', 'r1d2_lines',
       'r1d2_l_end', 'r1d2_p_start', 'r1d2_p_end', 'r1d3_l_start',
       'r1d3_lines', 'r1d3_l_end', 'r1d3_p_start', 'r1d3_p_end',
       'r1d4_l_start', 'r1d4_lines', 'r1d4_l_end', 'r1d4_p_start',
       'r1d4_p_end', 'r2d1_l_start', 'r2d1_lines', 'r2d1_l_end',
       'r2d1_p_start', 'r2d1_p_end', 'r2d2_l_start', 'r2d2_lines',
       'r2d2_l_end', 'r2d2_p_start', 'r2d2_p_end', 'r2d3_l_start',
       'r2d3_lines', 'r2d3_l_end', 'r2d3_p_start', 'r2d3_p_end',
       'r2d4_l_start', 'r2d4_lines', 'r2d4_l_end', 'r2d4_p_start',
       'r2d4_p_end', 'r3d1_l_start', 'r3d1_lines', 'r3d1_l_end',
       'r3d1_p_start', 'r3d1_p_end', 'r3d2_l_start', '

In [381]:
start_cols_pat = r'r\d+d\d+_p_start'
end_cols_pat = r'r\d+d\d+_p_end'

final_start_cols = []
final_end_cols = []

cols = list(apr_2024_info.columns)

for col in cols:
    if re.search(start_cols_pat,col):
        final_start_cols.append(col)
    elif re.search(end_cols_pat, col):
        final_end_cols.append(col)

zipped_cols = []
for start, end in zip(final_start_cols,final_end_cols):
    zipped_cols.append(start)
    zipped_cols.append(end)


info_cols = ['Author', 'role', 'Title']

final_cols = info_cols + zipped_cols 

apr_2024_breakdown = apr_2024_info[final_cols]

apr_2024_breakdown

,Author,role,Title,r1d1_p_start,r1d1_p_end,r1d2_p_start,r1d2_p_end,r1d3_p_start,r1d3_p_end,r1d4_p_start,...,r2d3_p_start,r2d3_p_end,r2d4_p_start,r2d4_p_end,r3d1_p_start,r3d1_p_end,r3d2_p_start,r3d2_p_end,r3d3_p_start,r3d3_p_end
0,Jeffrey R. Holland,Acting President of the Quorum of the Twelve A...,Motions of a Hidden Fire,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
1,J. Anette Dennis,First Counselor in the Relief Society General ...,Put Ye On the Lord Jesus Christ,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
2,Alexander Dushku,Of the Seventy,Pillars and Rays,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
3,Ulisses Soares,Of the Quorum of the Twelve Apostles,Covenant Confidence through Jesus Christ,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
4,Jack N. Gerard,Of the Seventy,Integrity: A Christlike Attribute,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
5,Henry B. Eyring,Second Counselor in the First Presidency,All Will Be Well Because of Temple Covenants,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
6,David A. Bednar,Of the Quorum of the Twelve Apostles,"“Be Still, and Know That I Am God”",1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
7,Massimo De Feo,Of the Seventy,Rise! He Calleth Thee,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
8,Brent H. Nielson,Of the Presidency of the Seventy,A Record of What I Have Both Seen and Heard,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
9,Jose L. Alonso,Of the Seventy,Jesus Christ at the Center of Our Lives,1,0,1,0,1,0,1,...,1,0,1,0,1,<NA>,<NA>,<NA>,<NA>,<NA>


In [382]:
path = 'D:\Faith and Religion Stuff\Come, Follow Me'
name = 'apr_2024_breakdown.csv'
full_path = os.path.join(path, name)
apr_2024_breakdown.to_csv(full_path,index = False)

Alright! Now that I've got this thing up and running, now I can start to functionalize it! 